In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import os

In [10]:
import cv2

import numpy as np

# ------------------- Set Paths -------------------
input_folder = "/content/drive/MyDrive/Porosity DL Project/Top"  # Folder containing original images
output_folder = "/content/drive/MyDrive/Porosity DL Project/Top/binarytop"         # Folder to save binary masks

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# ------------------- Process All Images -------------------
for filename in os.listdir(input_folder):
    if filename.endswith(('.tif')):  # Process only image files
        image_path = os.path.join(input_folder, filename)
        mask_path = os.path.join(output_folder, filename)  # Save with same filename

        # Load image in grayscale
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Apply Otsu's Thresholding
        _, mask = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Save binary mask
        cv2.imwrite(mask_path, mask)

        #print(f"Processed: {filename}")

print("All images processed and saved in:", output_folder)


KeyboardInterrupt: 

In [8]:
# Binary

# Path to your folder (update this)
folder_path = "/content/drive/MyDrive/Porosity DL Project/Top/cbt"

# Define image extensions
image_extensions = {".tif"}

# Count images
num_images = sum(1 for file in os.listdir(folder_path) if file.lower().endswith(tuple(image_extensions)))

print(f"Number of images in folder: {num_images}")

Number of images in folder: 813


In [4]:
import cv2
import numpy as np
import os

def crop_circle(image_path, save_path):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect circles using HoughCircles
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50, param1=50, param2=30, minRadius=20, maxRadius=500)

    if circles is not None:
        circles = np.uint16(np.around(circles))
        x, y, r = circles[0][0]  # Assume the largest detected circle

        # Crop the circular region
        mask = np.zeros_like(image, dtype=np.uint8)
        cv2.circle(mask, (x, y), r, (255, 255, 255), -1)

        # Apply the mask
        cropped = cv2.bitwise_and(image, mask)

        # Extract bounding box around the circle
        x1, y1, x2, y2 = x - r, y - r, x + r, y + r
        cropped = cropped[y1:y2, x1:x2]

        cv2.imwrite(save_path, cropped)
        return True

    return False

def process_images(drive_folder_path, save_folder):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    for filename in os.listdir(drive_folder_path):
        if filename.lower().endswith(('.tif')):
            image_path = os.path.join(drive_folder_path, filename)
            save_path = os.path.join(save_folder, filename)
            crop_circle(image_path, save_path)

drive_folder_path = "/content/drive/MyDrive/Porosity DL Project/Top/binarytop"
save_folder = "/content/drive/MyDrive/Porosity DL Project/Top/croppedbinarytop"

process_images(drive_folder_path, save_folder)
print("Cropping completed.")

KeyboardInterrupt: 

In [9]:
input_folder = "/content/drive/MyDrive/Porosity DL Project/Top/cbt"  # Folder containing binary images

# List to store results
results = []

# Loop through images
for filename in os.listdir(input_folder):
    if filename.endswith(".tif"):
        image_path = os.path.join(input_folder, filename)

        # Load binary image (0 = black, 255 = white)
        binary_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        # Get image dimensions
        height, width = binary_img.shape
        center = (width // 2, height // 2)
        radius = min(width, height) // 2  # Assuming the circle fits inside the square

        # Create a circular mask
        mask = np.zeros_like(binary_img)
        cv2.circle(mask, center, radius, 255, -1)  # Draw a white circle on a black background

        # Apply the mask to keep only circular region
        circular_region = cv2.bitwise_and(binary_img, mask)

        # Find contours in the circular region
        contours, _ = cv2.findContours(circular_region, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Initialize total pixels and black pixels in the circle
        total_pixels_in_circle = 0
        black_pixels_in_circle = 0

        # Iterate through contours
        for contour in contours:
          # Calculate the area of the contour
          area = cv2.contourArea(contour)

          # Consider only contours greater than 7 pixels
          if area > 7:
              # Get the bounding rectangle of the contour
              x, y, w, h = cv2.boundingRect(contour)

              # Count pixels within the bounding rectangle (more efficient than iterating through all pixels)
              total_pixels_in_circle += w*h
              black_pixels_in_circle += np.count_nonzero(circular_region[y:y+h, x:x+w] == 0)


        # Calculate porosity percentage (handle potential division by zero)
        if total_pixels_in_circle > 0:
            porosity = (black_pixels_in_circle / total_pixels_in_circle) * 100
        else:
            porosity = 0  # or handle the case appropriately

        # Store result
        results.append(porosity)

# ... (rest of your code)
# Compute statistics
count_images = len(results)
min_porosity = min(results)
max_porosity = max(results)
mean_porosity = sum(results) / count_images

# Print final statistics
print(f"Porosity Statistics:")
print(f" Total Images: {count_images}")
print(f" Min Porosity: {min_porosity:.2f}%")
print(f" Max Porosity: {max_porosity:.2f}%")
print(f" Mean Porosity: {mean_porosity:.2f}%")

Porosity Statistics:
 Total Images: 813
 Min Porosity: 13.42%
 Max Porosity: 27.69%
 Mean Porosity: 22.57%


In [10]:
import cv2
import numpy as np
import os
import pandas as pd

# ------------------- Set Paths -------------------
input_folder = "/content/drive/MyDrive/Porosity DL Project/Top/cbt"  # Folder containing binary images
output_excel = "/content/drive/MyDrive/Porosity DL Project/Top/cbt/Cbt.xlsx"  # Output Excel file path

# List to store results
results = []

# Loop through images
for filename in os.listdir(input_folder):
    if filename.endswith(".tif"):
        image_path = os.path.join(input_folder, filename)

        # Load binary image (0 = black, 255 = white)
        binary_img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

 # Get image dimensions
        height, width = binary_img.shape
        center = (width // 2, height // 2)
        radius = min(width, height) // 2  # Assuming the circle fits inside the square

        # Create a circular mask
        mask = np.zeros_like(binary_img)
        cv2.circle(mask, center, radius, 255, -1)  # Draw a white circle on a black background

        # Apply the mask to keep only circular region
        circular_region = cv2.bitwise_and(binary_img, mask)

        # Count total pixels inside the circle (both black and white)
        total_pixels_in_circle = np.count_nonzero(mask == 255)

        # Count black pixels (pores) inside the circle
        black_pixels_in_circle = np.count_nonzero((circular_region == 0) & (mask == 255))

        # Calculate porosity percentage
        porosity = (black_pixels_in_circle / total_pixels_in_circle) * 100

        # Store result
        results.append({
            "Image Name": filename,
            "Porosity (%)": porosity
        })

        # Extract position from filename
        positions = [int(entry["Image Name"][-8:-4]) for entry in results]
        porosity_values = [entry["Porosity (%)"] for entry in results]

        # Sort both based on positions
        positions, porosity_values = zip(*sorted(zip(positions, porosity_values)))

# Convert to DataFrame and save to Excel
df = pd.DataFrame(results)
df.to_excel(output_excel, index=False)
print(f"Results saved to: {output_excel}")

Results saved to: /content/drive/MyDrive/Porosity DL Project/Top/cbt/Cbt.xlsx


In [2]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 10.5 MB/s eta 0:00:00


In [3]:
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 60.8 MB/s eta 0:00:00
